In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
# from tensorflow.keras.datasets import cifar10
# import numpy as np

In [3]:
class AlexBlockA(keras.Model):
    def __init__(self, nf, kernel=(3, 3), cs=1, ps=2, input_shape=None, activation='relu', data_format='channels_first'):
        super(AlexBlockA, self).__init__(name='AlexBlockA')
        self.padding = layers.ZeroPadding2D(2, data_format=data_format)
        if input_shape and input_shape is not None: self.conv = layers.Conv2D(nf, kernel, strides=(cs, cs), input_shape=input_shape, activation=activation, data_format=data_format)
        else: self.conv = layers.Conv2D(nf, kernel, strides=(cs, cs), activation=activation, data_format=data_format)
        self.pool = layers.MaxPool2D(3, strides=ps, data_format=data_format)
    def call(self, x):
        x = self.padding(x)
        x = self.conv(x)
        x = self.pool(x)
        print('from alexa', x.shape)
        return x

In [4]:
class AlexBlockB(keras.Model):
    """
    nfs: filter for each conv layer
    kernels: kernel for each conv layer
    sts: stride for each conv layer
    """
    def __init__(self, nfs, kernels, sts, pool_size=(3, 3), pool_stride=1, activation='relu', padding='same'):
        super(AlexBlockB, self).__init__(name='AlexBlockB')
        if isinstance(nfs, int): nfs = [nfs for _ in range(3)]
        if isinstance(sts, int): sts = [sts for _ in range(3)]
        if len(kernels) == 1: kernels = [kernels[0] for _ in range(3)]
        self.conv = []
        for i in range(3):
            if i == 2: self.conv.append(layers.ZeroPadding2D(2))
            self.conv.append(layers.Conv2D(nfs[i], kernels[i], strides=sts[i], activation=activation))
        self.pool = layers.MaxPool2D(pool_size, strides=pool_stride, padding=padding)
    def call(self, x):
        for i in range(4):
            x = self.conv[i](x)
        x = self.pool(x)
        print('from alexb', x.shape)
        return x

In [5]:
class AlexNet(keras.Model):
    def __init__(self, input_shape, data_format='channels_first'):
        super(AlexNet, self).__init__(name='AlexNet')
        tf.keras.backend.set_image_data_format(data_format)
        self.body = keras.Sequential(name='Body')
        self.body.add(AlexBlockA(96, (11, 11), 4, 2, input_shape))
        self.body.add(AlexBlockA(256, (5, 5), 1, 2))
        self.body.add(AlexBlockB([384, 384, 256], [(3, 3)], [1, 1, 2]))
        self.fc1 = layers.Dense(4096, activation='relu')
        self.fc2 = layers.Dense(4096, activation='relu')
        self.fc3 = layers.Dense(1000, activation='softmax')
    def call(self, x):
        print(x.shape)
        x = self.body(x)
        x = layers.Flatten()(x)
        print('from flatten', x.shape)
        x = self.fc1(x)
        print('from dense 4096', x.shape)
        x = self.fc2(x)
        print('from dense 4096', x.shape)
        x = self.fc3(x)
        print('from dense 1000', x.shape)
        return x

In [6]:
alex = AlexNet(input_shape=(3, 224, 224))
alex.build((1, 3, 224, 224))
alex.summary()

NameError: name 'data_format' is not defined